In [1]:
import pandas as pd
import numpy as np
import pywebio.input as pi
import pywebio.output as po
from pywebio.utils import pyinstaller_datas
from communities.algorithms import louvain_method, girvan_newman, hierarchical_clustering, spectral_clustering

## 规范化拉普拉斯变换
相似度矩阵

In [2]:
def lap_mat(sim_mat):
    d = np.sum(sim_mat, axis=1).tolist()
    d = [i**(-0.5) for i in d]
    d_ = np.diag(d)
    an = len(sim_mat)
    mat_lap = np.eye(an) - np.matmul(np.matmul(d_,sim_mat), d_)
    return mat_lap

## 本证间隙法求最优聚类数
拉普拉斯矩阵

In [3]:
def main_gap(lap_matrix):
    mat, _ = np.linalg.eig(lap_matrix)
    mat = sorted(mat, reverse=True)
    g = [mat[i] - mat[i+1] for i in range(len(mat)-1)]
    print(g)
    for i in range(1, len(g)-1):
        if g[i]>g[i-1] and g[i]>g[i+1]:
            return i+1

## 模块度计算
邻接矩阵

In [4]:
def get_q(adj_m, c_, c_d):
    in_ = 0
    tot_ = 0
    in_list = []
    tot_list = []
    for i in c_d.keys():
        if i == c_:
            in_list.extend(c_d[i])
        tot_list.extend(c_d[i])
    for i in in_list:
        for z in in_list:
                in_ += adj_m[i][z]
    for i in in_list:
        for z in tot_list:
            tot_ += adj_m[i][z]
    return in_, tot_

In [5]:
def get_Q(adj_mat, c_set):
    m = 0
    adj_matrix_ = adj_mat - np.eye(len(adj_mat))
    for i in range(len(adj_matrix_)):
        for j in range(len(adj_matrix_[0])):
            m += adj_matrix_[i][j]
    m/=2
    len_ = 0
    for c in c_set:
        len_ += len(c)

    c_list = [0 for _ in range(len_)]
    for s in range(len(c_set)):
        for i in list(c_set[s]):
            c_list[i] = s
    c_dict = {}
    for i in range(len(c_list)):
        c_dict.setdefault(c_list[i],[])
        c_dict[c_list[i]].append(i)
    Q = 0
    for i in c_dict.keys():
        n,t = get_q(adj_matrix_, i, c_dict)
        Q += n/(2*m) - (t/(2*m))**2
    return Q

## louvain_method
邻接矩阵

In [6]:
def louvain_divide(adj_mat):
    community, _ = louvain_method(adj_mat)
    Q = get_Q(adj_mat, community)
    return community, Q

## girvan_newman
邻接矩阵

In [7]:
def girvan_newman_divide(adj_mat):
    community, _ = girvan_newman(adj_mat)
    Q = get_Q(adj_mat, community)
    return community, Q

## 分层聚类
邻接矩阵

In [8]:
def hierarchical_clustering_divide(adj_mat):
    community = hierarchical_clustering(adj_mat, linkage="complete")
    Q = get_Q(adj_mat, community)
    return community, Q

## 谱聚类
邻接矩阵

In [9]:
def spectral_clustering_divide(adj_mat):
    sim_matrix = adj_mat + np.eye(len(similar_matrix))
    lap_matrix = lap_mat(sim_matrix)
    k = main_gap(lap_matrix)
    community = spectral_clustering(adj_mat, k=k)
    Q = get_Q(adj_mat, community)
    return community, Q

## 最大Q

In [10]:
def max_q(adj_mat):
    method_list = [louvain_divide, girvan_newman_divide, hierarchical_clustering_divide, spectral_clustering_divide]
    com_dict = {i:list(method_list[i](adj_mat)) for i in range(len(method_list))}
    max_q_com = com_dict[0][0]
    max_q_ = com_dict[0][1]
    for i in com_dict.keys():
        if com_dict[i][1] > max_q_:
            max_q_ = com_dict[i][1]
            max_q_com = com_dict[i][0]
    return max_q_com, max_q_


## 方法字典

In [11]:
method_dict = {
    "louvain": louvain_divide,
    "girvan_newman": girvan_newman_divide,
    "分层聚类": hierarchical_clustering_divide,
    "谱聚类": spectral_clustering_divide,
    "最优": max_q
}

## 输入询问

In [12]:
def ask_info():
    data = pi.input_group(
        "模块划分",
        [
            pi.file_upload("相似度矩阵", name="file"),
            pi.select("模块划分方法", name="method", options=list(method_dict.keys()), value="最优")
        ],
    )
    return data["file"], data["method"]

## 输出结果

In [13]:
def put_com(community):
    po.put_text(str(community))

## 主函数

In [14]:
file,method = ask_info()
dataframe = pd.read_excel(file["content"], header=None)
similar_matrix = dataframe.values
adj_matrix = similar_matrix - np.eye(len(similar_matrix))
com, _ = method_dict[method](adj_matrix)
put_com(com)

[0.37689080145218623, 0.4026394431916033, 0.26550302116873, 0.2344969788312701]
